### Create prompt dataset

In [ ]:
! pip install -U sagemaker

In [ ]:
! pip install transformers

In [ ]:
import json

data = []
with open('data/all.jsonl') as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
f1 = open('data/all.jsonl')
Lines1 = f1.readlines()

quests, ans = [], []
print(len(Lines1))
for line in Lines1:
    row = json.loads(line)
    for answer in row["human_answers"]:
        quests.append("Prompt: "+row["question"])
        ans.append("Response: "+answer)
    for answer in row["chatgpt_answers"]:
        quests.append("Prompt: "+row["question"])
        ans.append("Response: "+answer)

print(f"{len(quests)}--{len(ans)}")

In [ ]:
import pandas as pd
import json
  
# Opening JSON file
f1 = open('data/all.jsonl')
Lines1 = f1.readlines()

questions, answers = [], []
print(len(Lines1))
for line in Lines1[:20000]:
    row = json.loads(line)
    for answer in row["human_answers"]:
        questions.append("Prompt: "+row["question"])
        answers.append("Response: "+answer)
    for answer in row["chatgpt_answers"]:
        questions.append("Prompt: "+row["question"])
        answers.append("Response: "+answer)

test_file = open("data/test.jsonl","w")
for line in Lines1[20000:]:
    test_file.write(line)

test_file.close()


df = pd.DataFrame()
df["question"] = questions
df["answer"] = answers
df = df.sample(frac = 1)
print(len(df.index))
df_train = df.iloc[:60000,:]
df_val = df.iloc[60000:,:]

df_train.to_csv("data/train.csv", index=False)
df_val.to_csv("data/val.csv", index=False)

### Upload data to S3

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch

In [ ]:
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
train_data_url = sess.upload_data(
    path="data/train.csv",
    key_prefix="promptsds",
)

valid_data_url = sess.upload_data(
    path="data/val.csv",
    key_prefix="promptsds",
)

In [ ]:
print(f"training file path {train_data_url}")
print(f"validation file path {valid_data_url}")

### Fine Tune FLAN T5 XXL (11b) on Seq2Seq

In [ ]:
base_job_name="sft-flanUl2-20b"

In [46]:
hyperparameters = {}
SM_DATA_DIR = "/opt/ml/input/data" 

hyperparameters["model_name_or_path"] = "google/flan-ul2"

hyperparameters["model_dir"] = f"/opt/ml/model"

hyperparameters["train_file"] = f"{SM_DATA_DIR}/train/train.csv"
hyperparameters["validation_file"] = f"{SM_DATA_DIR}/valid/val.csv"

hyperparameters["per_device_train_batch_size"] = 4
hyperparameters["per_device_eval_batch_size"] = 4
hyperparameters["block_size"] = 2048


hyperparameters["num_train_epochs"] = 4

In [47]:
smp_options = {
    "enabled":True,
    "parameters": {                        # Required
        "pipeline_parallel_degree": 1,     # Required
        "ddp": True,
        # parameters for sharded data parallelism
        "sharded_data_parallel_degree": 16,              # Add this to activate sharded data parallelism
        "partitions":1,
        "bf16":True,
        "skip_tracing": True

    }
}

mpi_options = {
    "enabled" : True,                      # Required
    "processes_per_host" : 8,              # Required
}

In [48]:
# launch with smp

estimator = PyTorch(
    base_job_name=base_job_name,
    source_dir="./scripts",
    entry_point="train.py",
    role=role,
    framework_version="1.13.1",
    py_version="py39", 
    instance_count=2,
    instance_type="ml.p4d.24xlarge",
    hyperparameters=hyperparameters,
    disable_profiler=True,
    debugger_hook_config=False,
    distribution={
        "smdistributed": {"modelparallel": smp_options},
        "mpi": mpi_options
    }
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [49]:
estimator.fit(inputs=data_channels)

[1,mpirank:8,algo-2]<stderr>:#015  3%|▎         | 109/3752 [16:05<8:54:09,  8.80s/it]
[1,mpirank:0,algo-1]<stdout>:[2023-05-02 02:51:12,046] [WARNING] [stage3.py:3068:step] 5 pytorch allocator cache flushes since last step. this happens when there is high memory pressure and is detrimental to performance. if this is happening frequently consider adjusting settings to reduce memory consumption. If you are unable to make the cache flushes go away consider adding torch.cuda.empty_cache() calls in your training loop to ensure that all ranks flush their caches at the same time
[1,mpirank:8,algo-2]<stderr>:#015  3%|▎         | 116/3752 [17:06<8:47:43,  8.71s/it]
[1,mpirank:0,algo-1]<stdout>:[2023-05-02 02:52:13,327] [WARNING] [stage3.py:3068:step] 5 pytorch allocator cache flushes since last step. this happens when there is high memory pressure and is detrimental to performance. if this is happening frequently consider adjusting settings to reduce memory consumption. If you are unable to mak

In [ ]:
estimator.fit({"train":train_data_url,"valid":valid_data_url})